# Tokenizers
Как вы помните, токенизация это процесс когда мы разбиваем текст на слова или части слов. Каждый токенайзер имеет внутри себя какую то таблицу, где на против id-шника стоит токен (слово или часть). И в процессе токенизации он разбивает текст на слова из своего словаря и затем меняет слово на его id. В итоге после токенизации мы получаем не текст а список id-шников.

Вопрос только как составить эту таблицу-словарь и как разбить текст на составляющие. Тем более что не во всех языках есть понятие пробела, да разбивать по пробелам и пунктуации не самый точный способ.

Мы уже знакомы с токенайзером от SpaCy и он был rule-based токенизатор, то есть основан на правилах. Так же мы использовали токенизаторы основанные на пробелах и пунктуации.

Какие минусы могут быть при разбиении текста по пробелам и пунктуации:  
- словарь может стать очень большим, так как слов очень много  
- сложно написать корректные правила разделения, где-то точка знак пунктуации, где часть смайла, где-то часть троеточия и таких нюансов тысячи
    
    
Сегодня углубимся в логику построения токенизаторов, посмотрим какие типы токенизаторов бывают и поймем как и где можно найти готовые токенизаторы. 

# Word tokenization
Если взять большой набор текстов, то вы получите огромное количество уникальных слов. Если вы хотите все их запомнить и обрабатвывать, то вам придется держать огромную таблицу внутри токенайзера, что скажется на скорости/памяти. Но даже не это страшно, вы будете копить в словаре все опечатки, а так же слова которые встречались раз или два. По таким редким словам у вас нет статистики и любая модель, которая будет подстраиваться под них будет учить явно не то что вы хотите от нее.

# Character tokenization
Проблему можно решить перейдя на символьный токенайзер. Новые символы появляются значительно реже чем слова.  
Но этот подход рождает свои проблемы:  
- текст становится очень длительной последовательностью, моделям уже очень сложно уловить зависимости в этой последовательности  
- эмбединги символов несут очень мало полезной информации для нас, в отличии от эмбедингов слов. 

**Токенайзеры не содеражат эмбединги! Они нужны для последующего использования в моделях.**

# Subword tokenization
Данный вид токенайзеров разбивает текст на токены, где токен может быть словом или частью слова.   
Принцип следующий - Если слово часто встречается, то мы запоминаем его целиков как отдельный токен. Если редко, то смотрим какие части слова встречаются наиболее часто и разбиваем его на них.  
Получается что решаем следующие проблемы:
- ушли от проблемы безконтрольного разрастания словаря
- наши токены представляют собой значимые кусочки информации, которые можно потом интерпретировать.
- можем обрабатывать даже те слова, которые никогда не видели!

## Byte-Pair encoding tokenization (BPE)
Особый вид токенизатора. Был предложен в 2015 году.
Алгоритм обучение токенизатора BPE состоял из двух шагов.
1. Одним из простых токенизаторов (пунктуация+пробел, правила) разбивали текст на слова. Это фаза pre-tokenization
2. Составляли частостную таблицу всех токенов, которые встретились в текстах при обучении словаря.
3. Формируем итоговый словарь токенизатора:
    - сначала добавляем в словарь все символы, которые встретились
    - затем смотрим на частоту совстречаемости тех токенов что уже есть в словаре и формируем из них новые токены, добавляя их в итоговый словарь. И так рекурсивно мы наполняем словарь, пока объем словаря не будет нас устраивать.  

Данный вид токенизаторов используется в GPT-2, Roberta.
[Пошаговая реализация алгоритма.](https://huggingface.co/course/chapter6/5?fw=pt)

In [ ]:
# Пример
("hug", 10), ("pug", 5), ("pun", 12), ("bun", 4), ("hugs", 5) - наша таблица после претокенизации

In [ ]:
vocabulary = ["b", "g", "h", "n", "p", "s", "u"] # добавили все символы

In [ ]:
# Пробуем разные разбиения слов по два символа и ищем самые часто встречающиеся.
"hu"  = 10 + 5 = 15
"un"  = 12 + 4 = 16
"ug" = 10 + 5 + 5 = 20 => vocabulary = ["b", "g", "h", "n", "p", "s", "u", "ug"]

In [ ]:
# Смотрим другие сочетания букв и заполняем в словарь.
# Когда все двубуквенные сочетания добавили, смотрим трехбуквенные и так далее

In [ ]:
# пусть словарь у нас ["b", "g", "h", "n", "p", "s", "u", "ug"], тогда следующие примеры разобьются на соответствующие токены:
"bug"  = ['b', 'ug']
"mug"  = ['<unk>', 'ug'] # m не в словаре, на практике такое не встретить, так как крупные корпуса текстов содераж все возможные символы

## WordPiece
Этот алгоритм токенизации используется в BERT, DistilBERT, и Electra. Алгоритм впервые появился в сети в 2012 году и очень похож на BPE.  
Алгоритм так же делает претокенизацию, инициализирует словарь всеми встретившимися символами и затем учит правила объединения символов в более общие токены. Только в отличие от BPE, где мы ориентировались на количество совстречаний, здесь мы максимизируем вероятность совместного использования токенов. То есть он смотрит на такую величину P(A,B) = P(A,B) / P(A) * P(B)  
[Пошаговая реализация алгоритма.](https://huggingface.co/course/chapter6/6?fw=pt)

## Unigram
Метод появился в 2018 году. Используется в ALBERT, XLNet.
Здесь все делается наоборот. Он сначала составляет огромный словарь из всех встреченных токенов в тексте, затем подстроки токенов.
А потом  начинает резать этот словарь по критерию, напоминащий WordPiece. Процедура учесения словаря останавливается когда его размер будет равен заданному параметру. Но словарь всегда будет содержать все символы, так что новые слова так же успешно будут разбиваться на токены.  
[Пошаговая реализация алгоритма.](https://huggingface.co/course/chapter6/7?fw=pt)

**Но как токенизировать текст состоящий из иероглифов, где нет пробелов?**  
Эту проблему можно решить если рассматривать текст как простую последовательность символов и тот же пробел считать за символ. Затем можно применить тот же BPE.

Ниже изображено сравнение алгоритмов.

![comparison](./tokenizer_comparison.png)

# Библиотеки transformers и tokenizers
Данная библиотеки является оберткой над всеми современными разработками. Они позволяют использовать модели и части пайплайна (такие как токенизатор) из топовых статей.

[Посмотреть какие модели на данный момент находятся в их хранилище можно здесь.](https://huggingface.co/models?sort=downloads)  
Перейдя в конкретную модель вы може ознакомиться с ней, посмотреть как установить ее, какие возможности она имеет.

**Давайте установим библиотеку transformers и воспользуемся ее встроенными средствами для токенизации.**

In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 22.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 36.8 MB/s eta 0:00:0000:0100:01


In [12]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.6 MB/s eta 0:00:00:00:010:01


In [138]:
from transformers import AutoTokenizer # Импортируем токенайзер

In [139]:
# Загрузим токенизатор из готовой модели, если она уже скачана, то она возьмет из локального хранилища
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased") 

In [140]:
tokenizer.tokenize("Вчера было солнечно.")

['В', '##че', '##ра', 'было', 'сол', '##не', '##чно', '.']

In [141]:
# Токенайзер хранит подслова как ##+токен. Так он сигнализирует что это не независимый токен/слово, а составная часть.
tokenizer.tokenize("it was sunny yesterday.") 

['it', 'was', 'sun', '##ny', 'ye', '##ster', '##day', '.']

In [142]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased") 

In [143]:
tokenizer.tokenize("it was sunny yesterday.")

['it', 'was', 'sunny', 'yesterday', '.']

In [144]:
tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")

In [146]:
# Этот токенайзер был обучен воспринимать пробел как обычный символ. 
# Поэтому если перед словом часто встречался пробел то они и запомнил его как набор симоволов с пробелом, обозначив его как _.
# Это как раз подойдет для иероглифов.
tokenizer.tokenize("it was sunny yesterday.")

['▁it', '▁was', '▁sunny', '▁yesterday', '.']

## Строим свой токенизатор
Каждый токенизатор в библиотеке `transformers` состоит из следующих составлющих:
1. нормализатор / normalization
2. претокенайзер / pre-tokenization
3. модель, правила / model
4. пост-обработка / post-processing
5. Опционально. Decoder - обратный процесс, из id-ников составить исходный текст 

![tokenization](./tokenization.png)

### Normalization
[Здесь можно посмотреть описание нормализаторов.](https://huggingface.co/docs/tokenizers/components)  
Перед тем как использовать алгоритмы токенизации WordPiece, BPE, Unigram необходимо подготовить текст - привести к нижнему регистру, очистить от лишних символов и акцентов.

In [149]:
from tokenizers import normalizers

In [150]:
dir(normalizers)

['BertNormalizer',
 'Lowercase',
 'NFC',
 'NFD',
 'NFKC',
 'NFKD',
 'NORMALIZERS',
 'Nmt',
 'Normalizer',
 'Precompiled',
 'Replace',
 'Sequence',
 'Strip',
 'StripAccents',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'normalizers',
 'unicode_normalizer_from_str']

In [151]:
from tokenizers.normalizers import (
    BertNormalizer, Lowercase, NFC, NFD, NFKC,
    StripAccents, Strip, Replace
)

In [152]:
# Создаем нормализатор из последовательности шагов
normalizer = normalizers.Sequence([
    normalizers.Replace("``", '"'),
    normalizers.Replace("''", '"'),
    normalizers.Replace("”", '"'),
    normalizers.Replace("“", '"'),
    normalizers.Replace('ˈ', "'"),
    normalizers.Replace('’',"'"),
    normalizers.Replace('–',"-"),
    normalizers.Replace('—',"-"),
    normalizers.Replace('−',"-"),
    normalizers.Replace('′',"'"),
    normalizers.Replace('⁄',"/"),
    NFD(), 
    StripAccents(), 
    Lowercase(), 
    Strip()
])

In [153]:
normalizer.normalize_str("Héllò hôw are ü?")

'hello how are u?'

### Pre-Tokenization
Претокенизация заключается в разделении текста на мелкие объекты, как правило по какому-то простому правилу и цепочке правил.   
Этот шаг позволяет уже примерно оценить выход токенизатора. Можно рассматривать этот шаг как деление на слова, которые потом могут разделиться на подслова.

[Здесь можно посмотреть описание пре-токенизаторов.](https://huggingface.co/docs/tokenizers/components)  

In [154]:
from tokenizers import pre_tokenizers

In [155]:
dir(pre_tokenizers)

['BertPreTokenizer',
 'ByteLevel',
 'CharDelimiterSplit',
 'Digits',
 'Metaspace',
 'PreTokenizer',
 'Punctuation',
 'Sequence',
 'Split',
 'UnicodeScripts',
 'Whitespace',
 'WhitespaceSplit',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'pre_tokenizers']

In [9]:
?pre_tokenizers.Metaspace

Init signature: pre_tokenizers.Metaspace(self, replacement='_', add_prefix_space=True)
Docstring:     
Metaspace pre-tokenizer

This pre-tokenizer replaces any whitespace by the provided replacement character.
It then tries to split on these spaces.

Args:
    replacement (:obj:`str`, `optional`, defaults to :obj:`▁`):
        The replacement character. Must be exactly one character. By default we
        use the `▁` (U+2581) meta symbol (Same as in SentencePiece).

    add_prefix_space (:obj:`bool`, `optional`, defaults to :obj:`True`):
        Whether to add a space to the first word if there isn't already one. This
        lets us treat `hello` exactly like `say hello`.
File:           /Users/u14510182/Documents/python_for_nlp_stud/venv/lib/python3.9/site-packages/tokenizers/pre_tokenizers/__init__.py
Type:           type
Subclasses:     


In [10]:
?pre_tokenizers.Digits

Init signature: pre_tokenizers.Digits(self, individual_digits=False)
Docstring:     
This pre-tokenizer simply splits using the digits in separate tokens

Args:
    individual_digits (:obj:`bool`, `optional`, defaults to :obj:`False`):
        If set to True, digits will each be separated as follows::

            "Call 123 please" -> "Call ", "1", "2", "3", " please"

        If set to False, digits will grouped as follows::

            "Call 123 please" -> "Call ", "123", " please"
File:           /Users/u14510182/Documents/python_for_nlp_stud/venv/lib/python3.9/site-packages/tokenizers/pre_tokenizers/__init__.py
Type:           type
Subclasses:     


In [156]:
pre_tokenizer = pre_tokenizers.Sequence([
    pre_tokenizers.Punctuation(),
    pre_tokenizers.Metaspace(replacement = '_', add_prefix_space = False), 
    pre_tokenizers.Digits(individual_digits=True)
])

In [157]:
pre_tokenizer.pre_tokenize_str("Hello! How are you? I'm fine, thank you. 123")

[('Hello', (0, 5)),
 ('!', (5, 6)),
 ('_How', (6, 10)),
 ('_are', (10, 14)),
 ('_you', (14, 18)),
 ('?', (18, 19)),
 ('_I', (19, 21)),
 ("'", (21, 22)),
 ('m', (22, 23)),
 ('_fine', (23, 28)),
 (',', (28, 29)),
 ('_thank', (29, 35)),
 ('_you', (35, 39)),
 ('.', (39, 40)),
 ('_', (40, 41)),
 ('1', (41, 42)),
 ('2', (42, 43)),
 ('3', (43, 44))]

In [158]:
pre_tokenizer.pre_tokenize_str("Call 911!")

[('Call', (0, 4)),
 ('_', (4, 5)),
 ('9', (5, 6)),
 ('1', (6, 7)),
 ('1', (7, 8)),
 ('!', (8, 9))]

### Model
Теперь нам необходимо выбрать какую модель токенайзера мы хотим сделать.  
[Здесь можно посмотреть описание доступных моделей.](https://huggingface.co/docs/tokenizers/components)  

In [159]:
from tokenizers import models as tok_models

In [160]:
model = tok_models.BPE(unk_token="[UNK]")
# model = tok_models.Unigram()
# model = tok_models.WordLevel()
# model = tok_models.WordPiece(unk_token="[UNK]")

In [16]:
?tok_models.Unigram

Init signature: tok_models.Unigram(self, vocab)
Docstring:     
An implementation of the Unigram algorithm

Args:
    vocab (:obj:`List[Tuple[str, float]]`, `optional`):
        A list of vocabulary items and their relative score [("am", -0.2442),...]
File:           /Users/u14510182/Documents/python_for_nlp_stud/venv/lib/python3.9/site-packages/tokenizers/models/__init__.py
Type:           type
Subclasses:     


In [17]:
?tok_models.WordPiece

Init signature: tok_models.WordPiece(self, vocab, unk_token, max_input_chars_per_word)
Docstring:     
An implementation of the WordPiece algorithm

Args:
    vocab (:obj:`Dict[str, int]`, `optional`):
        A dictionnary of string keys and their ids :obj:`{"am": 0,...}`

    unk_token (:obj:`str`, `optional`):
        The unknown token to be used by the model.

    max_input_chars_per_word (:obj:`int`, `optional`):
        The maximum number of characters to authorize in a single word.
File:           /Users/u14510182/Documents/python_for_nlp_stud/venv/lib/python3.9/site-packages/tokenizers/models/__init__.py
Type:           type
Subclasses:     


### Post-Processing

In [161]:
from tokenizers.processors import TemplateProcessing

In [19]:
?TemplateProcessing

Init signature: TemplateProcessing(self, single, pair, special_tokens)
Docstring:     
Provides a way to specify templates in order to add the special tokens to each
input sequence as relevant.

Let's take :obj:`BERT` tokenizer as an example. It uses two special tokens, used to
delimitate each sequence. :obj:`[CLS]` is always used at the beginning of the first
sequence, and :obj:`[SEP]` is added at the end of both the first, and the pair
sequences. The final result looks like this:

    - Single sequence: :obj:`[CLS] Hello there [SEP]`
    - Pair sequences: :obj:`[CLS] My name is Anthony [SEP] What is my name? [SEP]`

With the type ids as following::

    [CLS]   ...   [SEP]   ...   [SEP]
      0      0      0      1      1

You can achieve such behavior using a TemplateProcessing::

    TemplateProcessing(
        single="[CLS] $0 [SEP]",
        pair="[CLS] $A [SEP] $B:1 [SEP]:1",
        special_tokens=[("[CLS]", 1), ("[SEP]", 0)],
    )

In this example, each input sequence is iden

In [20]:
from tokenizers.processors import TemplateProcessing
post_processor = TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[("[CLS]", 1), ("[SEP]", 2)],
)

### Create tokenizer
С помошью созданных этапов собираем итоговый токенизатор.

In [162]:
from tokenizers import Tokenizer

In [163]:
our_tokenizer = Tokenizer(model)
our_tokenizer.normalizer = normalizer
our_tokenizer.pre_tokenizer = pre_tokenizer
# our_tokenizer.post_processor = post_processor

In [164]:
output = our_tokenizer.encode("Hello, y'all! How are you 😁 ?")
output.ids # пока не обучили, ничего не работает.

Exception: Unk token `[UNK]` not found in the vocabulary

### Обучаем токенизатор на данных Wiki-103

In [165]:
import re

In [167]:
def load_data(tp='train'):
    if tp not in ['train', 'test', 'valid']:
        raise Exception('ERROR: Wrong type of data.')
    
    pth = r"wikitext-103/" + f'wiki.{tp}.raw'
    heading_pattern = '\n (= ){1,}[^=]*[^=] (= ){1,}\n \n'
    with open(pth, 'r') as f:
        raw_text = f.read()
    
    raw_text = re.split(heading_pattern, raw_text)
    raw_text = [x.strip().strip('\n').strip() for x in raw_text if x and x not in [' ', '= ']]
    return raw_text

In [168]:
%%time
train_data = load_data('train')

CPU times: user 7.08 s, sys: 10.7 s, total: 17.8 s
Wall time: 20.5 s


In [169]:
len(train_data)

271821

In [170]:
train_data[0]

'Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more forgiving for series newcomers . Character designer Ra

In [171]:
def get_training_corpus():
    for i in range(0, 100000, 1000): # Не будем на всем объеме
        yield train_data[i : i + 1000]

In [172]:
SPEC_TOKENS = ["[UNK]", "[PAD]", "[MASK]"] 

In [179]:
trainer = trainers.BpeTrainer(
        vocab_size=100000, 
        min_frequency=0,
        show_progress=True,
        special_tokens=SPEC_TOKENS, 
        continuing_subword_prefix='##'
    )

In [180]:
%%time
our_tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)




CPU times: user 4min 48s, sys: 33.7 s, total: 5min 21s
Wall time: 38.8 s


### Методы токенизатора

In [ ]:
our_tokenizer.get_vocab()

In [182]:
our_tokenizer.get_vocab_size()

100000

In [183]:
our_tokenizer.id_to_token(234)

'ε'

In [184]:
our_tokenizer.token_to_id('_how')

5793

In [185]:
our_tokenizer.token_to_id('bla') # Такого токена не существует

In [99]:
?our_tokenizer.add_tokens

Signature: our_tokenizer.add_tokens(self, tokens)
Docstring:
Add the given tokens to the vocabulary

The given tokens are added only if they don't already exist in the vocabulary.
Each token then gets a new attributed id.

Args:
    tokens (A :obj:`List` of :class:`~tokenizers.AddedToken` or :obj:`str`):
        The list of tokens we want to add to the vocabulary. Each token can be either a
        string or an instance of :class:`~tokenizers.AddedToken` for more customization.

Returns:
    :obj:`int`: The number of tokens that were created in the vocabulary
Type:      builtin_function_or_method


### Encoding

In [186]:
encode = our_tokenizer.encode("Héllò hôw are ü?")

In [187]:
encode.tokens

['h', '##ello', '_how', '_are', '_u', '?']

In [188]:
encode.ids

[49, 13770, 5793, 5546, 5436, 34]

In [189]:
# Показывает с какой на какую позицую в исходной строке приходится i-ый токен
pos = encode.token_to_chars(1)
print('Position:', pos)
print('Token:', encode.tokens[1])
print('Original string part:', "Héllò hôw are ü?"[pos[0]: pos[1]])

Position: (1, 5)
Token: ##ello
Original string part: éllò


In [190]:
# Возвращает номер (не ID!!!) токена по позиции символа в строке
encode.char_to_token(4)

1

In [191]:
our_tokenizer.id_to_token(encode.ids[1])

'##ello'

### Decoding
Обратный процесс. Собираем из id-шников исходный текст.

In [192]:
from tokenizers import decoders

In [193]:
dir(decoders)

['BPEDecoder',
 'ByteLevel',
 'CTC',
 'Decoder',
 'Metaspace',
 'Sequence',
 'WordPiece',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'decoders']

In [195]:
our_tokenizer.decode(encode.ids) # Дефолтный декодер выглядит не супер.

'h ##ello _how _are _u ?'

In [196]:
decoder = decoders.Sequence([
    # decoders.BPEDecoder(suffix='##'),
    decoders.WordPiece(prefix='##'),
    decoders.Metaspace(replacement='_'),  
])
our_tokenizer.decoder = decoder

In [197]:
our_tokenizer.decode(encode.ids)

'hello  how  are  u?'

### Save/Load Tokenizer

In [198]:
our_tokenizer.save("Tokenizer_BPE100k.json")

In [199]:
loaded_our_tokenizer = Tokenizer.from_file("Tokenizer_BPE100k.json")

### Fast tokenizing
Мы можем зафиксировать наш токенайзер и обернуть в более быструю обертку.

In [126]:
from transformers import PreTrainedTokenizerFast

In [200]:
loaded_our_tokenizer_fast = PreTrainedTokenizerFast(
    tokenizer_file="Tokenizer_BPE100k.json",
    unk_token="[UNK]",
    pad_token="[PAD]",
    mask_token="[MASK]"
)